<a href="https://colab.research.google.com/github/caesarcc/python-tcc-url-fakenews-check/blob/main/jupyter/classificacao_passo02_treino_avaliacao.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Abrir no Colab"/></a>

## Treino e Avaliação do Modelo de Classificação

In [11]:
# Importação de bibliotecas utilizadas no treino e avaliação
import os
import torch
import random
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import cohen_kappa_score, accuracy_score, precision_recall_fscore_support, matthews_corrcoef
from transformers.file_utils import is_torch_available
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from IPython.display import display
%matplotlib inline

### Carrega dados processados no passo 1

In [2]:
pd.set_option("display.max_rows", 50, 'display.max_colwidth', 200)
dados = pd.read_csv('dados/fakebr_corpus_processado.csv', sep = ',')
dados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7074 entries, 0 to 7073
Data columns (total 7 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Unnamed: 0             7074 non-null   int64 
 1   texto                  7074 non-null   object
 2   classe                 7074 non-null   int64 
 3   texto_limpo            7074 non-null   object
 4   texto_processado       7074 non-null   object
 5   qtde_texto_limpo       7074 non-null   int64 
 6   qtde_texto_processado  7074 non-null   int64 
dtypes: int64(4), object(3)
memory usage: 387.0+ KB


In [3]:
dados[['classe','texto','texto_processado','qtde_texto_processado']].sample(n=3)

,classe,texto,texto_processado,qtde_texto_processado
1817,0,"Escândalos ameaçam colar no Brasil pecha de exportador de corrupção, diz ONG. Transparência Internacional divulgou relatório nesta quarta (25) sobre percepção da corrupção em todo o mundo; no docu...","Escândalos ameaçar colar Brasil pecha exportador corrupção , ONG . Transparência Internacional divulgar relatório ( 25 ) percepção corrupção mundo ; documentar , entidade destacar , , esforçar paí...",43
2330,0,"O bombeiro herói que há três décadas salva pessoas na praia recordista de afogamentos em SP. Sargento Gerson não sabe quantas pessoas já resgatou no Guarujá, mas diz que chega a fazer 10 atendimen...","bombeiro herói haver década salvo pessoa praia recordista afogamento SP . Sargento Gerson quanto pessoa resgatar Guarujá , chegar 10 atendimento dia alto temporada , 55 ano , pensar aposentadoria ...",253
2814,0,"Manifestantes protestam em Palmas pela absolvição do ex-presidente Lula. Segundo a organização, cerca de 70 pessoas participam da manifestação. A Polícia Militar não informou o número de participa...","Manifestantes protestar Palmas absolvição ex-presidente Lula . organização , cercar 70 pessoa participar manifestação . Polícia Militar informar participante . Manifestantes reunir frente ser Just...",156


### Geração de seed  
Com esta rotina consigo garantir a reprodução dos resultados mesmo que o ambiente for reiniciado
Aplicável às libs random, numpy e torch

In [4]:
def garantir_reprodutividade(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    if is_torch_available():
        torch.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)

garantir_reprodutividade(2)

### Carregando o modelo pré-treinado BERTimbau

In [5]:
model_name = "neuralmind/bert-base-portuguese-cased"
# máximo de tokens por frase
max_length = 350
# carregando o modelo
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=2)
# carregar tokenizador
tokenizer = BertTokenizer.from_pretrained(model_name, do_lower_case=False)

#Conforme mensagem o erro é esperado pois o modelo BertForSequence... está sendo inicializado por um BertForPreTraining.
# - This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

### Separando treino e validação

In [6]:
textos = dados['texto_processado'].tolist()
classes = dados['classe'].tolist()

textos_treino, textos_validacao, classes_traino, classes_validacao = train_test_split(
        textos, classes, test_size=0.2)

display(f"Treinamento: {len(classes_traino)}")
display(f"Validação: {len(classes_validacao)}")

'Treinamento: 5659'

'Validação: 1415'

### Tokenização e geração dos tensores

In [7]:
encodings_treino = tokenizer(textos_treino, truncation=True, padding=True, max_length=max_length)
encodings_validacao = tokenizer(textos_validacao, truncation=True, padding=True, max_length=max_length)

In [13]:
class TorchDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor([self.labels[idx]])
        return item

    def __len__(self):
        return len(self.labels)

# Converte as listas tokenizadas um dataset de tensonres
dataset_treino = TorchDataset(encodings_treino, classes_traino)
dataset_validacao = TorchDataset(encodings_validacao, classes_validacao)

### Melhoria do Treinamento do modelo BERTimbau (fine-tuning)

In [14]:
# Calcula todas métricas possíveis para analisar posteriormente
def calcula_metricas(pred):
  classes = pred.label_ids
  predicoes = pred.predictions.argmax(-1)
  precision, recall, f1, _ = precision_recall_fscore_support(classes, predicoes, average='binary')
  return {
      'accuracy': accuracy_score(classes, predicoes),
      'f1': f1,
      'precision': precision,
      'recall': recall,
      'mcc': matthews_corrcoef(classes, predicoes),
      # cálcula o coeficiente kappa (nível de concordância ou reprodutibilidade)
      'kappa': cohen_kappa_score(classes, predicoes),
  }

### Hiperparâmetros confome monografia e documentação do HuggingFace

In [15]:
# Para rodar no drive é necessário adicionar ao output_dir: drive/MyDrive/Colab Notebooks
hiper_parametros = TrainingArguments(
    output_dir="/modelos/fake_url_bertimbau",
    overwrite_output_dir=True,
    num_train_epochs=5,             
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=50,                 
    weight_decay=0.01,               
    evaluation_strategy='no',
    logging_dir='./logs'            
)

trainer = Trainer(
    model=model,
    args=hiper_parametros,
    train_dataset=dataset_treino,
    eval_dataset=dataset_validacao,
    tokenizer=tokenizer,
    compute_metrics=calcula_metricas
)

In [16]:
# Treina o modelo
trainer.train()

***** Running training *****
  Num examples = 5659
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3540


  0%|          | 0/3540 [00:00<?, ?it/s]

KeyboardInterrupt: 

### Validação do Modelo tunado

In [ ]:

metricas = trainer.evaluate()
acc = metricas['eval_accuracy']
f1 = metricas['eval_f1']
precision = metricas['eval_precision']
recall = metricas['eval_recall']
mcc = metricas['eval_mcc']
kappa = metricas['kappa']

### Validando o modelo

In [ ]:
plt.plot(history['train_acc'], label='train accuracy')
plt.plot(history['val_acc'], label='validation accuracy')

plt.title('Training history')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend()
plt.ylim([0, 1])

print(classification_report(y_test, y_pred, target_names=class_names))

def show_confusion_matrix(confusion_matrix):
  hmap = sns.heatmap(confusion_matrix, annot=True, fmt="d", cmap="Blues")
  hmap.yaxis.set_ticklabels(hmap.yaxis.get_ticklabels(), rotation=0, ha='right')
  hmap.xaxis.set_ticklabels(hmap.xaxis.get_ticklabels(), rotation=30, ha='right')
  plt.ylabel('True sentiment')
  plt.xlabel('Predicted sentiment');

cm = confusion_matrix(y_test, y_pred)
df_cm = pd.DataFrame(cm, index=class_names, columns=class_names)
show_confusion_matrix(df_cm)

### Salvando o melhor modelo

In [ ]:
model.save_pretrained("/modelos/fake_url_bertimbau/melhor_modelo")